# SIMILARITA CON BERT 

In [ ]:
from entity_linking_script import find_most_similar_pairs, read_first_column

file1_path = "../csv_file/off_ricette_normalizzato_da_pipeline.csv"
file2_path = "../csv_file/off_normalizzato_a_mano.csv"

list1 = read_first_column(file_path=file1_path)
list2 = read_first_column(file2_path)

most_similar_pairs = find_most_similar_pairs(list1, list2)

print("Most similar couples:\n")
for item1, item2, score in most_similar_pairs:
    print(f"({item1}) --- ({item2}) --- Similarity: {score:.3f}")

# Approccio ispirato all'università di Bari

In [ ]:
from entity_linking_script import (
    RecipeTransformer,
    compute_embeddings,
    find_similar_by_title,
    read_first_column,
)
from tqdm import tqdm

file1_path = "../csv_file/off_ricette_normalizzato_da_pipeline.csv"
file2_path = "../csv_file/off_normalizzato_a_mano.csv"

list1 = read_first_column(file1_path)
list2 = read_first_column(file2_path)

# Initialize the transformer
transformer_name = "davanstrien/autotrain-recipes-2451975973"
transformer = RecipeTransformer(transformer_name)

# Compute embeddings for all recipes in list2 (to reduce execution time)
print("Calculating embeddings for list2...")
embeddings2 = compute_embeddings(list2, transformer)

# Create a list of tuples (index, title) for list2
entities_list2 = list(enumerate(iterable=list2))

# Find the most similar recipe for each item in list1
most_similar_pairs = []
print("Searching for the most similar recipes...")
for recipe_title in tqdm(list1, desc="Similarity search"):
    similar_recipe, similarity_score = find_similar_by_title(
        recipe_title, entities_list2, embeddings2, transformer
    )
    most_similar_pairs.append((recipe_title, similar_recipe[1], similarity_score))

# Output the results
print("Most similar recipe pairs found:\n")
for item1, item2, score in most_similar_pairs:
    print(f"({item1}) --------- ({item2}) --------- Similarity: {score:.3f}")

# APPROCCIO IBRIDO SFRUTTANDO INDICATOR

### Estraggo titolo, calorie, grassi, carboidrati e proteine delle ricette di hummus e OFF

In [4]:
from entity_linking_script import (
    find_k_most_similar_pairs_with_indicators,
    normalize_columns,
    read_specified_columns,
)

file1_path = "../csv_file/pp_recipes_rows.csv"
column1 = [
    "title",
    "totalFat [g]",
    "totalCarbohydrate [g]",
    "protein [g]",
    "servingSize [g]",
]
list1 = read_specified_columns(file1_path, column1, delimiter=",")
list1 = normalize_columns(list1)

file2_path = "../csv_file/off_rows.csv"
column2 = ["product_name", "fat_100g", "carbohydrates_100g", "proteins_100g"]
list2 = read_specified_columns(file2_path, column2, delimiter="\t")


for lst1 in (list1, list2):  # Cambiamo il nome della variabile da `list` a `lst`
    for i, item in enumerate(lst1):
        # Converti la tupla in una lista
        temp_item = list(
            item
        )  # Usa la funzione incorporata list() per creare una lista

        # Modifica l'elemento: aggiungi "aaa" all'inizio e sposta il primo elemento originale alla fine
        temp_item.insert(0, "aaa")  # Aggiunge normalized_name all'inizio
        temp_item.append(temp_item.pop(1))  # Sposta l'attuale primo elemento alla fine

        # Aggiorna l'elemento nella lista principale (riconvertendolo in tupla, se necessario)
        lst1[i] = tuple(temp_item)


most_similar_pairs = find_k_most_similar_pairs_with_indicators(list1, list2, k=1)

print("Coppie più simili trovate:\n")
for item1, item2, score, original_name1, original_name2 in most_similar_pairs:
    print(
        f"({item1})({original_name1}) --------- ({item2})({original_name2}) --------- Similarità: {score:.3f}"
    )

Coppie più simili trovate:

(aaa)(Cranberry-Orange Caramel Corn) --------- (aaa)( moutarde au moût de raisin ) --------- Similarità: 1.040
(aaa)(Cocktail De Camarones) --------- (aaa)(Sauce Sweety chili 0%) --------- Similarità: 1.037
(aaa)(Inbal Jerusalem Hotel Snowball) --------- (aaa)(Sauce Sweety chili 0%) --------- Similarità: 1.039
(aaa)(Santorini Roasted Leg of Lamb) --------- (aaa)(Sauce Sweety chili 0%) --------- Similarità: 1.032
(aaa)(Cantonese Chicken Burgers) --------- (aaa)( moutarde au moût de raisin ) --------- Similarità: 1.040


### Test of various methods

In [ ]:
from entity_linking_script import (
    find_k_most_similar_pairs_with_indicators,
    find_most_similar_pairs,
)

list1 = [("Pasta", 30, 5, 10), ("Pane", 50, 1, 10)]
list2 = [("Riso", 40, 2, 8), ("Pizza", 20, 10, 12)]
result = find_k_most_similar_pairs_with_indicators(list1, list2)
print(result)

list1 = [("Pasta", 100, 0, 0), ("Pane", 0, 0, 0)]
list2 = [("Riso", 0, 2, 8), ("Pizza", 0, 50, 50)]
result = find_k_most_similar_pairs_with_indicators(list1, list2)
print(result)

list1 = [("Pasta", 33, 33, 33), ("Pane", 0, 0, 0)]
list2 = [("Riso", 0, 2, 8), ("Pizza", 33, 33, 33)]
result = find_k_most_similar_pairs_with_indicators(list1, list2)
print(result)

list1 = ["pasta", "pane"]
list2 = ["riso", "pizza"]
result = find_most_similar_pairs(list1, list2)
print(result)